In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [9]:
# Load the dataset
NP_Data = pd.read_csv('Downloads/NP DATASET.csv')

In [10]:
NP_Data.head()

,Organism Name,Compound Name,SMILES,Source
0,Momordica Charantia,Quercetin,Oc1cc(O)c2C(=O)C(=C(Oc2c1)c3ccc(O)c(O)c3)O,NPASS
1,Momordica Charantia,Niacin,OC(=O)c1cccnc1,NPASS
2,Momordica Charantia,Retinol,CC(=C\CO)/C=C/C=C(C)/C=C/C1=C(C)CCCC1(C)C,NPASS
3,Momordica Charantia,Ascorbate,OC[C@H](O)[C@H]1OC(=O)[C-](O)C1=O,NPASS
4,Momordica Charantia,Vitamin E,CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@]1(C)CCc2c(...,NPASS


In [11]:
NP_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2151 entries, 0 to 2150
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organism Name  1121 non-null   object
 1   Compound Name  1121 non-null   object
 2   SMILES         1121 non-null   object
 3   Source         1121 non-null   object
dtypes: object(4)
memory usage: 67.3+ KB


### <center> Feature Extraction

In [12]:
import re
from rdkit import Chem
from rdkit.Chem import AllChem
import networkx as nx

In [13]:
def smiles_tokenizer(smiles):
    """
    Tokenize SMILES string using regex pattern.
    
    Args:
        smiles (str): SMILES string to tokenize
        
    Returns:
        list: List of tokens
    """
    pattern = r'(\[[^\]]+\]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])'
    tokens = re.findall(pattern, smiles)
    return tokens

In [14]:
def smiles_to_molecular_graph(smiles):
    """
    Convert SMILES string to molecular graph representation.
    
    Args:
        smiles (str): SMILES string
        
    Returns:
        dict: Dictionary containing:
            - 'mol': RDKit molecule object
            - 'graph': NetworkX graph
            - 'node_features': Dictionary of node features (atomic number, degree, etc.)
            - 'edge_features': Dictionary of edge features (bond types)
            - 'adjacency_matrix': Numpy adjacency matrix
    """
    # Parse SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")
    
    AllChem.Compute2DCoords(mol)
    
    # NetworkX graph
    G = nx.Graph()
    
    # Nodes
    node_features = {}
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        G.add_node(idx)
        node_features[idx] = {
            'atomic_num': atom.GetAtomicNum(),
            'symbol': atom.GetSymbol(),
            'degree': atom.GetDegree(),
            'formal_charge': atom.GetFormalCharge(),
            'hybridization': str(atom.GetHybridization()),
            'is_aromatic': atom.GetIsAromatic(),
            'num_hydrogens': atom.GetTotalNumHs()
        }
    
    # Edges
    edge_features = {}
    for bond in mol.GetBonds():
        start = bond.GetBeginAtomIdx()
        end = bond.GetEndAtomIdx()
        G.add_edge(start, end)
        edge_features[(start, end)] = {
            'bond_type': str(bond.GetBondType()),
            'is_conjugated': bond.GetIsConjugated(),
            'is_aromatic': bond.GetIsAromatic()
        }
        # Reverse edge for undirected graph
        edge_features[(end, start)] = edge_features[(start, end)]
    
    # Adjacency matrix
    n_atoms = mol.GetNumAtoms()
    adj_matrix = np.zeros((n_atoms, n_atoms))
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        adj_matrix[i, j] = 1
        adj_matrix[j, i] = 1
    
    return {
        'mol': mol,
        'graph': G,
        'node_features': node_features,
        'edge_features': edge_features,
        'adjacency_matrix': adj_matrix
    }

In [15]:
def safe_smiles_to_graph(smiles):
    """
    Safely convert SMILES to molecular graph with error handling.
    
    Args:
        smiles (str): SMILES string
        
    Returns:
        dict or None: Molecular graph dictionary if successful, None if error
    """
    try:
        return smiles_to_molecular_graph(smiles)
    except Exception as e:
        print(f"Error processing SMILES: {smiles[:50]}... - {str(e)}")
        return None

In [17]:
from tqdm import tqdm
tqdm.pandas()

NP_Data['SMILES Tokens'] = NP_Data['SMILES'].progress_apply(smiles_tokenizer)
NP_Data['Molecular Graph'] = NP_Data['SMILES'].progress_apply(safe_smiles_to_graph)

 52%|█████▏    | 1122/2151 [00:00<00:00, 15070.76it/s]


TypeError: expected string or bytes-like object, got 'float'